In [ ]:
#2015년도 흥행작 top30 네이버 영화 코드 입력 + 영화이름 추가
from selenium import webdriver
from bs4 import BeautifulSoup
import pandas as pd

movie_codes = [115977,121048,98438]
review_list=[]
# 영화 id 수만큼 id 반복
for code in movie_codes:
    url_1 = 'https://movie.naver.com/movie/bi/mi/pointWriteFormList.nhn?code='+str(code)
    url_2 = '&type=after&isActualPointWriteExecute=false&isMileageSubscriptionAlready=false&isMileageSubscriptionReject=false&onlyActualPointYn=Y&page='
    url_m = 'https://movie.naver.com/movie/bi/mi/basic.naver?code=' + str(code)
    num = 0 
    
     # 새로운 창 안뜨는 옵션
    chrome_options = webdriver.ChromeOptions()
    chrome_options.add_argument('—headless')
    chrome_options.add_argument('—no-sandbox')
    chrome_options.add_argument('—disable-dev-shm-usage')
    driver = webdriver.Chrome('chromedriver.exe', options=chrome_options)
    driver.get(url_m)
    html = driver.page_source
    soup = BeautifulSoup(html,'html.parser')
    movie_name = soup.select("div.mv_info_area > div.mv_info > h3 > a")[0].text
    
    # 1부터 10페이지까지 돌림
    for page in range(1,11):
        options = webdriver.ChromeOptions()
        options.add_argument("headless")
        driver = webdriver.Chrome('chromedriver.exe', options=chrome_options)
        url = url_1+url_2+f'{page}'
        driver.get(url)
        html = driver.page_source
        soup = BeautifulSoup(html,'html.parser')
        
        # 각 페이지 별 review (각 페이지당 댓글 10개)
        for i in range(10):
            review = soup.find('span',{'id':f'_filtered_ment_{i}'})
            review = review.get_text().strip()
            score = soup.select("div.star_score > em")[i].text.strip()
            
            num += 1
            
            review_list.append([num, movie_codes,int(score)/2, review])

            
            
df = pd.DataFrame(review_list, columns=["num","영화코드","평점","댓글"])
print(df)

df.to_csv("naver_movie_review.csv", encoding='utf-8-sig', index=False)